In [10]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import time

from sklearn.metrics import mean_squared_error

In [79]:
def warp_img(img):
    # https://nikolasent.github.io/opencv/2017/05/07/Bird's-Eye-View-Transformation.html
    img_h = img.shape[0]
    img_w = img.shape[1]
    
    # src = np.float32([[0, img_h], [img_w, img_h], [0, img_h // 5], [img_w, img_h // 5]])
    # dst = np.float32([[0, img_h], [img_w, img_h], [0, 0], [img_w, 0]])
    
    src = np.float32([[0, img_h], [img_w, img_h], [0, img_h // 10], [img_w, img_h // 10]])
    dst = np.float32([[569, img_h], [711, img_h], [0, 0], [img_w, 0]])
    
    M = cv2.getPerspectiveTransform(src, dst) # The transformation matrix
    
    img = img[500:(img_h), 0:img_w] # Apply np slicing for ROI crop
    img = cv2.warpPerspective(img, M, (img_w, img_h)) # Image warping
    img = img[0:img_h-150, 350:900]
    return img


def apply_region_of_interest(img):
    img_h = img.shape[0]
    img_w = img.shape[1]
    
    return img[500:(img_h), 0:img_w]

def apply_gaussian_blur(img, kernel_size=(5, 5)):
    return cv2.GaussianBlur(img, kernel_size, 0)

def apply_canny_edge(img, low_threshold=100, high_threshold=200):
    return cv2.Canny(img, low_threshold, high_threshold)

def apply_dilation(img, kernel_size=(3, 3)):
    return cv2.dilate(img, np.ones(kernel_size, np.uint8))

def detect_hough_lines(img):
    return cv2.HoughLinesP(
        img,
        rho=1,
        theta=np.pi/180,
        threshold=20,
        minLineLength=2,
        maxLineGap=5
    )

def draw_hough_lines(img, lines):
    for line in lines:
        for x1, y1, x2, y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), 255, 5)
    return img

def find_largest_contour(img):
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return max(contours, key=cv2.contourArea, default=None)

def draw_rectangle_features(img, rect, throttle, steer):
    width = min(rect[1][1], rect[1][0])
    box = np.intp(cv2.boxPoints(rect))
    top_left = box[np.argmax(box[:, 1])] - np.array([0, 50])
    bottom_left = box[np.argmin(box[:, 1])] - np.array([0, 50])
    
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
    cv2.drawContours(img, [box], 0, (255, 0, 0), 2)
    
    cv2.circle(img,
                center=tuple(map(int, rect[0])),
                radius=5,
                color=(0, 0, 255),
                thickness=5)
    
    steer_result = f"a:{rect[2]:.2f}, s:{steer:.2f}"
    throttle_result = f"w:{width:.2f},t:{throttle:.2f}"
    cv2.putText(img,
                text=steer_result,
                org=tuple(map(int, bottom_left)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(255, 0, 0),
                thickness=2)
    cv2.putText(img,
                text=throttle_result,
                org=tuple(map(int, top_left)),
                fontFace=cv2.FONT_HERSHEY_SIMPLEX,
                fontScale=1,
                color=(255, 0, 0),
                thickness=2)
    # cv2.putText(img,
    #             text=str(round(steer, 2)),
    #             org=tuple(map(lambda x: int(x - 50), rect[0])),
    #             fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    #             fontScale=1,
    #             color=(255, 0, 0),
    #             thickness=2)
    # cv2.putText(img,
    #             text=str(round(width, 2)),
    #             org=(bottom_left[0] - 50, bottom_left[1]),
    #             fontFace=cv2.FONT_HERSHEY_SIMPLEX, map(int, rect[0])
    #             fontScale=1,
    #             color=(255, 0, 0),
    #             thickness=2)
    # cv2.putText(img,
    #             text=str(round(throttle, 2)),
    #             org=(bottom_left[0] - 50, bottom_left[1] + 50),
    #             fontFace=cv2.FONT_HERSHEY_SIMPLEX,
    #             fontScale=1,
    #             color=(255, 0, 0),
    #             thickness=2)
    return img

def map_values(rect:tuple) -> (float, float):
    """
    Map the values for the steer to (-1, 1)
    and the values for the throttle to (0, 1)
    
    Parameters
    ----------
    rect: tuple
        the rectangle that is used to determine the throttle and steering angle
    Return
    ------
    throttle: float
        the throttle for the car
    steer: float
        the steering angle for the car
    """
    # A lot of work is still needed here
    center = rect[0]
    d1 = rect[1][0]
    d2 = rect[1][1]
    width = min(rect[1][1], rect[1][0])
    height = max(rect[1][1], rect[1][0])
    angle = rect[2]
    
    # rounding to the nearest 5
    width = int(5 * round(width/5))
    angle = int(5 * round(angle/5))
    
    if angle in (0, 90, -0, -90):
        angle = 0
    
    elif d1 < d2:
        angle = 90 - angle
    
    else:
        angle = - angle

    throttle = width / 110
    steer = angle / 90
    
    return throttle, steer

In [80]:
def show_process_image(img):
    img_w, img_h = img.shape[1], img.shape[0]
    img = warp_img(img)
    
    img_blur = apply_gaussian_blur(img.copy())
    img_canny = apply_canny_edge(img_blur)
    img_canny = apply_dilation(img_canny)
    
    lines = detect_hough_lines(img_canny.copy())
    img_hou = np.zeros((img_h, img_w), dtype=np.uint8)
    draw_hough_lines(img_hou, lines)
    
    biggest_rectangle = find_largest_contour(img_hou)
    if biggest_rectangle is None:
        return
    
    rect = cv2.minAreaRect(biggest_rectangle)
    
    throttle, steer = map_values(rect)
    
    img = draw_rectangle_features(img, rect, throttle, steer)
    return img, img_blur, img_hou

def process_image(img):
    img_w, img_h = img.shape[1], img.shape[0]
    img = warp_img(img)
    
    img_blur = apply_gaussian_blur(img.copy())
    img_canny = apply_canny_edge(img_blur)
    img_canny = apply_dilation(img_canny)
    
    lines = detect_hough_lines(img_canny.copy())
    img_hou = np.zeros((img_h, img_w), dtype=np.uint8)
    draw_hough_lines(img_hou, lines)
    
    biggest_rectangle = find_largest_contour(img_hou)
    if biggest_rectangle is None:
        return
    
    rect = cv2.minAreaRect(biggest_rectangle)
    
    angle = rect[2]
    throttle, steer = map_values(rect)
    return steer, throttle

In [81]:
folder = "imgs"
images = []
imgs_canny = []
imgs_hou = []

true_throttles = []
true_steers = []

pred_throttles = []
pred_steers = []

for img_name in os.listdir(folder):
    split_name = img_name.split("_")
    true_steer = int(split_name[1])
    true_throttle = int(split_name[3])
    
    img = cv2.imread(os.path.join(folder, img_name), cv2.IMREAD_GRAYSCALE)
    # img, img_blur, img_hou = show_process_image(img)
    steer, throttle = process_image(img)
    
    true_steers.append(true_steer)
    true_throttles.append(true_throttle)
    pred_steers.append(steer)
    pred_throttles.append(throttle)
    # print(f"Steer: {steer:.2f}, \tThrottle: {throttle:.2f}")
    # images.append(img)
    # imgs_canny.append(img_canny)
    # imgs_hou.append(img_hou)

In [82]:
true_steers = [x / 20 for x in true_steers]
true_throttles = [x / 30 for x in true_throttles]


# calculate the RMSE
rmse_steer = mean_squared_error(true_steers, pred_steers, squared=False)
rmse_throttle = mean_squared_error(true_throttles, pred_throttles, squared=False)

print(f"RMSE Steer: {rmse_steer:.2f}")
print(f"RMSE Throttle: {rmse_throttle:.2f}")

RMSE Steer: 0.08
RMSE Throttle: 0.48


In [83]:
pred_steers

[-0.8888888888888888,
 -0.8888888888888888,
 -0.8888888888888888,
 -0.8888888888888888,
 -0.8888888888888888,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.8888888888888888,
 0.8888888888888888,
 0.8888888888888888,
 0.8888888888888888,
 0.8888888888888888,
 0.8888888888888888,
 0.9444444444444444,
 0.8888888888888888,
 0.8888888888888888,
 0.8888888888888888]

In [84]:
true_steers

[-1.0,
 -1.0,
 -1.0,
 -1.0,
 -1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]

In [85]:
true_throttles

[0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.5,
 0.5,
 0.5,
 0.5,
 0.5,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666,
 0.16666666666666666]

In [86]:
pred_throttles

[0.3181818181818182,
 0.3181818181818182,
 0.3181818181818182,
 0.3181818181818182,
 0.3181818181818182,
 0.9090909090909091,
 0.8181818181818182,
 0.9090909090909091,
 0.9090909090909091,
 0.9090909090909091,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.5454545454545454,
 0.3181818181818182,
 0.36363636363636365,
 0.3181818181818182,
 0.3181818181818182,
 0.3181818181818182,
 0.6818181818181818,
 0.6818181818181818,
 0.18181818181818182,
 0.18181818181818182,
 0.18181818181818182]